# Chat Streaming in Java

In reactive programming with Project Reactor, ensuring that a Flux stream processes elements in a sequential (linear) order is typically the default behavior. However, certain operations can potentially alter the order, especially when dealing with parallel processing or asynchrony.

If you want to ensure that operations are performed in a strict sequence, you can use operators like `concatMap` , `flatMapSequential` , or simply rely on the default behavior of `map` and `flatMap` when not dealing with parallelism.

Here's an example demonstrating how to process a `Flux` stream in a sequential order using `concatMap` :

<h2>Example: Sequential Processing with</h2> `concatMap`

In [ ]:
import reactor.core.publisher.Flux;
import reactor.core.publisher.Mono;

public class SequentialFluxExample {

    public static void main(String[] args) {
        Flux<String> flux = Flux.just("one", "two", "three", "four");

        flux.concatMap(SequentialFluxExample::processElement)
            .doOnNext(System.out::println)
            .blockLast();  // Block to wait for completion in this example
    }

    private static Mono<String> processElement(String element) {
        return Mono.just(element)
                .map(e -> {
                    // Simulate some processing
                    return "Processed " + e;
                });
    }
}


<h2>Explanation:</h2>

<ul>
    <li><code>Flux.just("one", "two", "three", "four")</code>: Creates a <code>Flux</code> emitting a sequence of strings.</li>
    <li><code>concatMap(SequentialFluxExample::processElement)</code>: Ensures that each element is processed in the order they are emitted. The <code>processElement</code> method is called for each element, and it returns a <code>Mono&lt;String&gt;</code>.</li>
    <li><code>processElement(String element)</code>: Simulates processing by appending "Processed" to each element.</li>
    <li><code>doOnNext(System.out::println)</code>: Prints each processed element to the console.</li>
    <li><code>blockLast()</code>: Blocks the main thread until the <code>Flux</code> completes. This is typically used in examples or tests to wait for the reactive pipeline to complete.</li>
</ul>

<h2>Sequential Processing with <code>flatMapSequential</code></h2>

<p>Another way to ensure sequential processing is by using <code>flatMapSequential</code>. 
This operator ensures that even if the inner publishers are running asynchronously, the results are emitted in the original order.</p>

<h3>Example: Sequential Processing with <code>flatMapSequential</code></h3>

In [ ]:
import reactor.core.publisher.Flux;
import reactor.core.publisher.Mono;

public class SequentialFluxExample {

    public static void main(String[] args) {
        Flux<String> flux = Flux.just("one", "two", "three", "four");

        flux.flatMapSequential(SequentialFluxExample::processElement)
            .doOnNext(System.out::println)
            .blockLast();  // Block to wait for completion in this example
    }

    private static Mono<String> processElement(String element) {
        return Mono.just(element)
                .map(e -> {
                    // Simulate some processing
                    return "Processed " + e;
                });
    }
}


<h3>Explanation:</h3>
<ul>
    <li><code>flatMapSequential</code>: Ensures that the results are emitted in the original order, even if the processing of elements is done asynchronously.</li>
    <li>Both <code>concatMap</code> and <code>flatMapSequential</code> can be used to maintain the sequential order of a <code>Flux</code> stream, ensuring that elements are processed and emitted in the order they were received.</li>
    <li>Use <code>concatMap</code> for strictly sequential processing and <code>flatMapSequential</code> when you need to handle asynchronous inner publishers while maintaining the order.</li>
</ul>

*************************************

************************

<p>To implement the code to ensure the <code>Flux</code> stream processes elements in a sequential (linear) order, we can use different approaches such as <code>concatMap</code>, <code>flatMapSequential</code>, and the default sequential processing with <code>map</code>.</p>

<p>Here are three different ways to achieve sequential processing:</p>

<h3>1. Using <code>concatMap</code></h3>

In [ ]:
package com.smarttrak.openai.controller;

import org.springframework.ai.chat.client.ChatClient;
import org.springframework.beans.factory.annotation.Autowired;
import org.springframework.http.MediaType;
import org.springframework.web.bind.annotation.GetMapping;
import org.springframework.web.bind.annotation.RequestParam;
import org.springframework.web.bind.annotation.RestController;
import reactor.core.publisher.Flux;
import reactor.core.publisher.Mono;

@RestController
public class OpenAiController {

    private final ChatClient chatClient;

    @Autowired
    public OpenAiController(ChatClient.Builder chatClientBuilder) {
        this.chatClient = chatClientBuilder.build();
    }

    @GetMapping(value = "/ai-stream-inline-concat", produces = MediaType.TEXT_EVENT_STREAM_VALUE)
    public Flux<String> generationConcatMap(@RequestParam("input") String userInput) {
        return chatClient.prompt()
                .user(userInput)
                .stream()
                .content()
                .concatMap(this::processElement)  // Ensures sequential processing
                //.delayElements(Duration.ofMillis(500))  // Adjust the delay for slow streaming
                .onErrorResume(e -> Flux.just("An error occurred while generating AI content: " + e.getMessage()));
    }

    private Mono<String> processElement(String element) {
        return Mono.just(element)
                .map(e -> "Processed " + e);
    }
}


<h3>2. Using <code>flatMapSequential</code></h3>

In [ ]:
package com.smarttrak.openai.controller;

import org.springframework.ai.chat.client.ChatClient;
import org.springframework.beans.factory.annotation.Autowired;
import org.springframework.http.MediaType;
import org.springframework.web.bind.annotation.GetMapping;
import org.springframework.web.bind.annotation.RequestParam;
import org.springframework.web.bind.annotation.RestController;
import reactor.core.publisher.Flux;
import reactor.core.publisher.Mono;

@RestController
public class OpenAiController {

    private final ChatClient chatClient;

    @Autowired
    public OpenAiController(ChatClient.Builder chatClientBuilder) {
        this.chatClient = chatClientBuilder.build();
    }

    @GetMapping(value = "/ai-stream-inline-flatmap", produces = MediaType.TEXT_EVENT_STREAM_VALUE)
    public Flux<String> generationFlatMapSequential(@RequestParam("input") String userInput) {
        return chatClient.prompt()
                .user(userInput)
                .stream()
                .content()
                .flatMapSequential(this::processElement)  // Ensures sequential processing
                //.delayElements(Duration.ofMillis(500))  // Adjust the delay for slow streaming
                .onErrorResume(e -> Flux.just("An error occurred while generating AI content: " + e.getMessage()));
    }

    private Mono<String> processElement(String element) {
        return Mono.just(element)
                .map(e -> "Processed " + e);
    }
}


<h3>3. Using Default Sequential Processing with <code>map</code></h3>

In [ ]:
package com.smarttrak.openai.controller;

import org.springframework.ai.chat.client.ChatClient;
import org.springframework.beans.factory.annotation.Autowired;
import org.springframework.http.MediaType;
import org.springframework.web.bind.annotation.GetMapping;
import org.springframework.web.bind.annotation.RequestParam;
import org.springframework.web.bind.annotation.RestController;
import reactor.core.publisher.Flux;

@RestController
public class OpenAiController {

    private final ChatClient chatClient;

    @Autowired
    public OpenAiController(ChatClient.Builder chatClientBuilder) {
        this.chatClient = chatClientBuilder.build();
    }

    @GetMapping(value = "/ai-stream-inline-map", produces = MediaType.TEXT_EVENT_STREAM_VALUE)
    public Flux<String> generationMap(@RequestParam("input") String userInput) {
        return chatClient.prompt()
                .user(userInput)
                .stream()
                .content()
                .map(element -> "Processed " + element)  // Default sequential processing
                //.delayElements(Duration.ofMillis(500))  // Adjust the delay for slow streaming
                .onErrorResume(e -> Flux.just("An error occurred while generating AI content: " + e.getMessage()));
    }
}


<h3>Explanation:</h3>
<ul>
    <li><strong>Using <code>concatMap</code>:</strong> Ensures each element is processed one by one in the order they are emitted.</li>
    <li><strong>Using <code>flatMapSequential</code>:</strong> Ensures the results are emitted in the original order, even if processing is asynchronous.</li>
    <li><strong>Using Default Sequential Processing with <code>map</code>:</strong> Directly processes each element sequentially without transforming it into another reactive type.</li>
</ul>
<p>These examples ensure that the elements are processed in a sequential (linear) order, and they provide different ways to handle the processing based on your specific needs.</p>

<h3>Example: Introducing Delay in SSE Stream</h3>

In [ ]:
package com.smarttrak.openai.controller;

import org.springframework.ai.chat.client.ChatClient;
import org.springframework.beans.factory.annotation.Autowired;
import org.springframework.http.MediaType;
import org.springframework.web.bind.annotation.GetMapping;
import org.springframework.web.bind.annotation.RequestParam;
import org.springframework.web.bind.annotation.RestController;
import reactor.core.publisher.Flux;
import reactor.core.publisher.Mono;

import java.time.Duration;

@RestController
public class OpenAiController {

    private final ChatClient chatClient;

    @Autowired
    public OpenAiController(ChatClient.Builder chatClientBuilder) {
        this.chatClient = chatClientBuilder.build();
    }

    @GetMapping(value = "/ai-stream-inline-delay", produces = MediaType.TEXT_EVENT_STREAM_VALUE)
    public Flux<String> generationWithDelay(@RequestParam("input") String userInput) {
        return chatClient.prompt()
                .user(userInput)
                .stream()
                .content()
                .concatMap(this::processElement)  // Ensures sequential processing
                .delayElements(Duration.ofMillis(500))  // Introduce delay in the stream
                .onErrorResume(e -> Flux.just("An error occurred while generating AI content: " + e.getMessage()));
    }

    private Mono<String> processElement(String element) {
        return Mono.just(element)
                .map(e -> "Processed " + e);
    }
}


<h3>Explanation:</h3>
<ul>
    <li><code>concatMap(this::processElement)</code>: Ensures that each element is processed sequentially.</li>
    <li><code>delayElements(Duration.ofMillis(500))</code>: Introduces a delay of 500 milliseconds between each emitted element in the stream. Adjust the duration as needed.</li>
</ul>

<h3>Result:</h3>
With this implementation, the elements in the stream will be emitted with a delay of 500 milliseconds between each element. This delay is introduced using the delayElements operator from Project Reactor, which works seamlessly with MediaType.TEXT_EVENT_STREAM_VALUE .

*****************************************************

***********************************************

# Kafka Integration in Chat Streaming
Configuring AWS Managed Streaming for Apache Kafka (MSK) and AWS Command Line Interface (AWS CLI) in a Spring Boot application while adhering to design patterns and principles can be a comprehensive task. Below is a modular approach that leverages design patterns and best practices:

RECOMMENDED - https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/home.html

## 1. Project Structure
First, let's define a modular project structure:

In [ ]:
src/main/java/com/example/awsmsk
    ├── config
    │   ├── AwsMskConfig.java
    │   ├── AwsCliConfig.java
    ├── service
    │   ├── KafkaProducerService.java
    │   ├── KafkaConsumerService.java
    ├── controller
    │   ├── KafkaController.java
    ├── model
    │   ├── Message.java
    ├── Application.java


## 2. Dependency Management
Add the necessary dependencies in your pom.xml :

In [ ]:
<dependencies>
    <!-- Spring Boot and Kafka dependencies -->
    <dependency>
        <groupId>org.springframework.boot</groupId>
        <artifactId>spring-boot-starter</artifactId>
    </dependency>
    <dependency>
        <groupId>org.springframework.kafka</groupId>
        <artifactId>spring-kafka</artifactId>
    </dependency>
    <!-- AWS SDK dependencies -->
    <dependency>
        <groupId>software.amazon.awssdk</groupId>
        <artifactId>kafka</artifactId>
    </dependency>
    <dependency>
        <groupId>software.amazon.awssdk</groupId>
        <artifactId>aws-sdk-java</artifactId>
    </dependency>
</dependencies>


## 3. Configuration Classes
<b>AwsMskConfig.java</b>

This configuration class sets up the Kafka producer and consumer:

In [ ]:
package com.example.awsmsk.config;

import org.apache.kafka.clients.consumer.ConsumerConfig;
import org.apache.kafka.clients.producer.ProducerConfig;
import org.apache.kafka.common.serialization.StringDeserializer;
import org.apache.kafka.common.serialization.StringSerializer;
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.kafka.core.*;

import java.util.HashMap;
import java.util.Map;

@Configuration
public class AwsMskConfig {

    @Bean
    public ProducerFactory<String, String> producerFactory() {
        Map<String, Object> config = new HashMap<>();
        config.put(ProducerConfig.BOOTSTRAP_SERVERS_CONFIG, "your-msk-bootstrap-servers");
        config.put(ProducerConfig.KEY_SERIALIZER_CLASS_CONFIG, StringSerializer.class);
        config.put(ProducerConfig.VALUE_SERIALIZER_CLASS_CONFIG, StringSerializer.class);
        return new DefaultKafkaProducerFactory<>(config);
    }

    @Bean
    public KafkaTemplate<String, String> kafkaTemplate() {
        return new KafkaTemplate<>(producerFactory());
    }

    @Bean
    public ConsumerFactory<String, String> consumerFactory() {
        Map<String, Object> config = new HashMap<>();
        config.put(ConsumerConfig.BOOTSTRAP_SERVERS_CONFIG, "your-msk-bootstrap-servers");
        config.put(ConsumerConfig.GROUP_ID_CONFIG, "group_id");
        config.put(ConsumerConfig.KEY_DESERIALIZER_CLASS_CONFIG, StringDeserializer.class);
        config.put(ConsumerConfig.VALUE_DESERIALIZER_CLASS_CONFIG, StringDeserializer.class);
        return new DefaultKafkaConsumerFactory<>(config);
    }

    @Bean
    public ConcurrentKafkaListenerContainerFactory<String, String> kafkaListenerContainerFactory() {
        ConcurrentKafkaListenerContainerFactory<String, String> factory = new ConcurrentKafkaListenerContainerFactory<>();
        factory.setConsumerFactory(consumerFactory());
        return factory;
    }
}


## AwsCliConfig.java

This configuration class sets up the AWS CLI:

In [ ]:
package com.example.awsmsk.config;

import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import software.amazon.awssdk.auth.credentials.ProfileCredentialsProvider;
import software.amazon.awssdk.services.s3.S3Client;

@Configuration
public class AwsCliConfig {

    @Bean
    public S3Client s3Client() {
        return S3Client.builder()
                .credentialsProvider(ProfileCredentialsProvider.create())
                .build();
    }
}


## 4. Service
<b>KafkaProducerService.java</b>

This service handles producing messages to Kafka:

In [ ]:
package com.example.awsmsk.service;

import org.springframework.beans.factory.annotation.Autowired;
import org.springframework.kafka.core.KafkaTemplate;
import org.springframework.stereotype.Service;

@Service
public class KafkaProducerService {

    private final KafkaTemplate<String, String> kafkaTemplate;

    @Autowired
    public KafkaProducerService(KafkaTemplate<String, String> kafkaTemplate) {
        this.kafkaTemplate = kafkaTemplate;
    }

    public void sendMessage(String topic, String message) {
        kafkaTemplate.send(topic, message);
    }
}


## KafkaConsumerService.java

This service handles consuming messages from Kafka:

In [ ]:
package com.example.awsmsk.service;

import org.springframework.kafka.annotation.KafkaListener;
import org.springframework.stereotype.Service;

@Service
public class KafkaConsumerService {

    @KafkaListener(topics = "your-topic", groupId = "group_id")
    public void consume(String message) {
        System.out.println("Consumed message: " + message);
    }
}


## 5. Controller

<b>KafkaController.java</b>

This controller exposes endpoints to send messages to Kafka:

In [ ]:
package com.example.awsmsk.controller;

import com.example.awsmsk.service.KafkaProducerService;
import org.springframework.beans.factory.annotation.Autowired;
import org.springframework.web.bind.annotation.*;

@RestController
@RequestMapping("/api/kafka")
public class KafkaController {

    private final KafkaProducerService producerService;

    @Autowired
    public KafkaController(KafkaProducerService producerService) {
        this.producerService = producerService;
    }

    @PostMapping("/publish")
    public void sendMessageToKafkaTopic(@RequestParam("message") String message) {
        producerService.sendMessage("your-topic", message);
    }
}


In [ ]:
## 6. Model

<b>Message.java<b>
A simple model class:

In [ ]:
package com.example.awsmsk.model;

public class Message {
    private String content;

    // getters and setters
}


## 7. Application Class

<b>Application.java</b>

The entry point of the Spring Boot application:

In [ ]:
package com.example.awsmsk;

import org.springframework.boot.SpringApplication;
import org.springframework.boot.autoconfigure.SpringBootApplication;

@SpringBootApplication
public class Application {

    public static void main(String[] args) {
        SpringApplication.run(Application.class, args);
    }
}


## Summary

<ui>Configuration Classes: Separate classes for AWS MSK and AWS CLI configurations.<br>
Services: Dedicated services for producing and consuming Kafka messages.<br>
Controller: A REST controller to expose Kafka endpoints.<br>
Design Patterns: Dependency Injection (DI) for configuration and services.<br>
Modular Approach: Each component has a clear responsibility.<br></ui>

This setup ensures a clean, modular, and maintainable architecture while leveraging Spring Boot's capabilities and adhering to design principles.